In [1]:
import os
import pickle

import requests
from flask import Flask, request, jsonify

from pymongo import MongoClient


MODEL_FILE = os.getenv('MODEL_FILE', 'lin_reg.bin')
MONGODB_ADDRESS = os.getenv('MONGODB_ADDRESS', 'mongodb://127.0.0.1:27017')
EVIDENTLY_SERVICE_ADDRESS = os.getenv('EVIDENTLY_SERVICE_ADDRESS', 'http://127.0.0.1:5000')

In [2]:
with open(MODEL_FILE,'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [3]:
app = Flask('Duration')

mongo_client = MongoClient(MONGODB_ADDRESS)

db = mongo_client.get_database('prediction_service')
collection = db.get_collection('data')

In [4]:
@app.route('/predict', methods = ['POST'])
def predict():
    
    ride = request.get_json()
    
    record = {}
    record['PU_DO'] = '{:}_{:}'.format(ride.get('PULocation'), ride.get('DOLocation'))
    
    X = dv.transform([record])
    y_pred = lr.predict(X)
    
    result = {
        'duration':float(y_pred)
    }
    
    save_to_db(record, y_pred) # prediction -> y_pred
    save_to_evidently_service(record, y_pred) # prediction ->y_pred
    
    return jsonify(result)

def save_to_db(record, prediction):
    rec = record.copy()
    rec['prediction'] = prediction
    
    collectin.insert_one(rec)

def save_to_evidently_service(record, prediction):
    rec = record.copy()
    rec['prediction'] = prediction
    requests.post(f'{EVIDENTLY_SERVICE_ADDRESS}/iterate/taxi', json(rec))

if __name__ == '__main__':
    app.run(debug = True, host = '0.0.0.0', port = 9696)

 * Serving Flask app "Duration" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses.
 * Running on http://10.129.0.20:9696/ (Press CTRL+C to quit)
 * Restarting with watchdog (inotify)
Traceback (most recent call last):
  File "/home/kkr/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/kkr/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.initialize(argv)
  File "/home/kkr/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 88, in inner
    return method(app, *args, **kwargs)
  File "/home/kkr/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 632, in initialize
    self.init_sockets()
  File "/home/kkr/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 282, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/home/kkr/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 229, in _bind_s

SystemExit: 1

/home/kkr/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [43]:
ride = {}
ride['PULocation']= 11
ride['DOLocation']= 21
features = {}
features['PU_DO'] = '{:}_{:}'.format(ride.get('PULocation'), ride.get('DOLocation'))

X = dv.transform([features])
float(lr.predict(X))

9.53392026179451

In [5]:
%save -f my_app 1-4

PermissionError: [Errno 13] Permission denied: 'my_app.py'